In [3]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import lxml.html
from lxml import etree
import xml.etree.ElementTree as ET
import string
import csv
import numpy as np
import time
import socket
import re
import pandas as pd

# Cluster Labeling
improvement of foxlink

## 1° Feature: images and calculate weights

In [27]:
def farFromAnchestor(leaf, ancestor, counter):
  if counter>=0 and leaf != None:
    
    if str(leaf.tag) == ancestor:
      return leaf
    
    counter=counter-1
    parent=leaf.getparent()
    return farFromAnchestor(parent,ancestor,counter)

def find_images(cluster, list_img):
    for page in cluster:
        #ritorna la get
        response = requests.get(page)
        #ritorna l'elemento html della pagina
        tree = lxml.html.fromstring(response.text)
        #ritorna la lista dei nodi foglia
        imgs = tree.xpath('//img[not(child::*)]')
        #trovo il dominio della pagina
        domain = re.search('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})',page).group()
        
        for elem in imgs:
          #list of element
          list_elem = []
          a = farFromAnchestor (elem,'a',5) #cerca il nodo che ha un link
          if a is not None: #effettua un filtraggio
            href=a.get("href")
            if href is not None:
                if (href.startswith('/')):
                    href = domain + href
                if not (href.endswith(".html") or "http" in href  or "contact" in href or "compare" in href 
                    or "www." in href or href.endswith(".php") or "twitter" in href or "facebook" in href 
                    or href == '/' or href == '#' or "email" in href or href.endswith(".htm")):
                    #print("Link: " + href +"\t"+str(child.get("src"))+"\n")
                    list_elem.append(page)
                    list_elem.append(str(elem.get("src")))
                    list_elem.append(0)
                    list_img.append(list_elem)

          elif a is None: #elemento che non ha link che porta in altre pagine
            
            #print('File: ' + str(elem.get("alt"))+ "\t" + str(elem.get("src")))
            list_elem.append(page)
            list_elem.append(str(elem.get("src")))
            list_elem.append(0)
            list_img.append(list_elem)
    
def weight_images(list_images, num_pages):
    for image in list_images:
        f = 1
        url = image[0]
        src = image[1]
        url_visti = [url]
        for new_img in list_images:
            if new_img[0] not in url_visti and new_img[1] == src:
                f += 1
                url_visti.append(new_img[0])
        
        image[2] = 1 - (f/num_pages)
        
def start_first_feature(cluster):
    start = time.time()
    # [[url_pagina, src, peso], ... ]
    list_images = [] 
    #trova le immagini che non hanno link che portano in altre pagine e inseriscile nella list_images
    find_images(cluster, list_images)
    #calcolo del peso del cluster in base alle immagini trovate e alla loro frequenza
    weight_images(list_images, len(cluster))
    
    pesoTot = 0.0
    for image in list_images:
        pesoTot += image[2]
    stop = time.time()
    print("Time:", round((stop - start)/60, 3), ' min') #time
    #print('Peso del cluster:', pesoTot)
    return pesoTot

def first_feature(domain_cluster):
    for domain in domain_cluster.keys():
        clusters = domain_cluster[domain]
        if(len(clusters) == 2):
            print("---- Domain:", domain, "----")
            print("- 1° Cluster")
            Peso_c1 = start_first_feature(clusters[0]["url_list"])
            print("- 2° Cluster")
            Peso_c2 = start_first_feature(clusters[1]["url_list"])
        
            if Peso_c1 > Peso_c2:
                clusters[0]["label"] = "Prodotto"
                clusters[1]["label"] = "Catalogo"
            else:
                clusters[1]["label"] = "Prodotto"
                clusters[0]["label"] = "Catalogo"

## 2° Feature: Information of the table and description of product with title

In [44]:
#FEATURE GENERALE:

#Cerchiamo di scovare una table fatta da div
  
#output:[[nodo1, fratello1 ... fratello n],[nodo 2, fratello 1 ... fratelo n]]
def allBrotherOfHtml(tree):
    d={}
    elements=tree.xpath("//*")
    for element1 in elements:
        if element1.getparent() not in d.keys():
            values=[element1]
            d[element1.getparent()]=values
        for element2 in elements:
            if not element1 == element2:
                if element1.getparent()== element2.getparent():         
                      if element2 not in d[element1.getparent()]:
                            d[element1.getparent()].append(element2)
    return d

  
def sameAttribute(list_html_elem, attribute):
  count=0
  for elem in list_html_elem:
    if elem.get(attribute) == None:
      count=0
    elif elem.get(attribute) == list_html_elem[0].get(attribute):
      count=count+1
  
  if count/len(list_html_elem)>=0.8:
    return True
  return False
  

def isAnchestor(child, target, ancestor):  
    if str(child.tag)==str(ancestor.tag):
      return False   
    if str(child.tag )== target :
      return True
    
    return isAnchestor(child.getparent(),target,ancestor)

def featureTables(table):

  values=table.xpath('.//*')
  result=True
  
  for value in values:
    if value is None:
      result=False
    if isAnchestor(value,'button',table) or isAnchestor(value,'a',table) or isAnchestor(value,'use',table) or isAnchestor(value,'input',table) :
      result=False
  return result


def calcoloPeso(title, text):
  list_title = title.lower().split(" ")
  count=0
  text=text.lower()
  
  for parola in list_title:
    if len(parola)<2:
      list_title.remove(parola)
      
  denominatore=len(list_title)

  for parola in list_title:
    if (',' or ';' or ':') in parola:
      parola=parola.replace(',', '')
      parola=parola.replace(';', '')
      parola=parola.replace(':', '')

    if parola in text:
      count+=1      
  return count/len(list_title)

def start_second_Feature(link):
    #ritorna la get
    response = requests.get(link)
    #ritorna l'elemento html della pagina
    tree = lxml.html.fromstring(response.text)
    #verifichiamo con una booleana se la pagina è un prodotto
    prod = False

    d=allBrotherOfHtml(tree)

    titles=tree.xpath("//title")
    if len(titles)>0:
        title=titles[0]
        #print("Titolo --> ", title.text)

    if title.text is not None:
        totalresult=''
        for parent in d.keys():
          if len(d[parent])>=2:
            if sameAttribute(d[parent],'class'):
              if featureTables(parent):
                leafs = parent.xpath('.//child::text()[not(child::*)]')
                result=''
                for leaf in leafs:
          
                  if leaf is not None:
                    if not (leaf.strip()==''):
                      if not ( '{ ' in leaf ):
                        result=result +str(leaf)+"\n"
                        totalresult=totalresult +str(leaf)+"\n"
                if not (result.strip() == ''):
                  if (calcoloPeso(title.text, result)>=0.4):
                    prod = True
                    #print("\n\n Superla la soglia, è prodotto "+ str(calcoloPeso(title.text, result))) 
        if (calcoloPeso(title.text, totalresult)>=0.8):
            prod = True
            #print("potrebbe essere Prodotto")

        return prod
    else:
        return prod
    
def second_feature(domain_cluster):
    for domain in domain_cluster.keys():
        clusters = domain_cluster[domain]
        if(len(clusters) == 1):
            print("Domain:", domain)
            start = time.time()
            contProd = 0
            for link in clusters[0]["url_list"]:
                if start_second_Feature(link):
                    contProd += 1
        
            if contProd > 0:
                clusters[0]["label"] = "Prodotto"
            else:
                clusters[0]["label"] = "Catalogo"
            
            stop = time.time()
            print("Tempo:", round((stop - start)/60, 3), ' min') #time
    

# Support Method for Accuracy

In [5]:
#Deve essere inserito il path dei due file da confrontare in formato csv
def metric(table1, table2):
    d1 = create_dict(table1)
    d2 = create_dict(table2)
    return 'Accuracy is: '+str(accuracy(d1, d2))+'%'

def create_dict(table):
    d = {}
    reader = csv.reader(open(table, 'rt'))
    for row in reader:
        d[str(row[0])+str(row[1])] = str(row[2])
    return d

def accuracy(d1, d2):
    esatti=0
    tot=0
    for k in d1.keys():
        if k in d2.keys():
            if d1[k]==d2[k]:
                esatti+=1
            tot+=1
    return (esatti/tot)*100

## Load Data

In [30]:
def find_domain(pos,stringa):
    domain = ""
    while(stringa[pos] != "'"):
        domain += stringa[pos]
        pos += 1
    return pos, domain

def find_shingle(pos, stringa):
    shingle = []
    #all'inizio punto al primo elemento della lista
    while(stringa[pos] != "]"):
        if stringa[pos] == "," and stringa[pos+1] == " ":
            pos += 2
        elif stringa[pos] == "-":
            numero = int(stringa[pos] + stringa[pos+1])
            shingle.append(numero)
            pos += 2
        else:
            numero = int(stringa[pos])
            shingle.append(numero)
            pos += 1
    return pos, shingle #ritorno la parentesi quadra o la virgola (+1)

def find_url_list(pos, stringa):
    url_pages = []
    #all'inizio punto sulla u del link che mi interessa
    #chiude con )]) e poi secondo shingle
    while(stringa[pos] != "]"):
        url = ""
        if stringa[pos] == "u" and stringa[pos+1] == "'":
            pos += 2
            while(stringa[pos] != "'"):
                url += stringa[pos]
                pos += 1
            
            #print("url:",url)
            
            url_pages.append(url)
            pos += 2 #spazio e scorri la tripla
            while(stringa[pos] != ")"):
                pos += 1 #scorro

            if stringa[pos+1] != "]":
                pos += 4 # , (u
            else:
                pos += 1 #punto alla quadra chiusa che finisce la lista di triple
    pos += 1 #punto alla parentesi chiusa
    return pos, url_pages

def split_cluster(pos, stringa):
    #inizializzo lista dei cluster e cluster
    list_cluster = []
    cluster1 = { "shingle": [], "url_list" : [], "label" : ""}
    #puntiamo sulla parentesi quadra dello shingle
    pos += 3
    #punto al primo elemento dello shingle
    pos, shingle = find_shingle(pos, stringa)
    print("- Find Shingle 1:", shingle)
    cluster1["shingle"] = shingle
    #avanzo finchè non arrivo alla prima u della tripla
    pos += 5
    pos, url_list = find_url_list(pos, stringa)
    #inseriamo i dati ricavati nel dizionario
    
    print("- Find the url of cluster 1")
    
    cluster1["url_list"] = url_list
    list_cluster.append(cluster1)
    #ora puntiamo alla parentesi tonda, verifichiamo se abbiamo finito o se
    #abbiamo un secondo shingle
    if stringa[pos+1] != "]":
        #creiamo il secondo cluster
        cluster2 = { "shingle": [], "url_list" : [], "label" : ""}
        #muoviamo la posizione fino al primo elemento dello shingle
        pos += 5
        #punto al primo elemento dello shingle
        pos, shingle = find_shingle(pos, stringa)
        
        print("- Find Shingle 2:", shingle)
        
        cluster2["shingle"] = shingle
        #avanzo finchè non arrivo alla prima u della tripla
        pos += 5
        pos, url_list = find_url_list(pos, stringa)
        #inseriamo i dati ricavati nel dizionario
        print("- Find the url of cluster 2")
        cluster2["url_list"] = url_list
        list_cluster.append(cluster2)
    return list_cluster

def init_find(stringa):
    pos = 0
    if stringa[pos] == "(" and stringa[pos+1] == "u" and stringa[pos+2] == "'":
        pos += 3
        pos, domain = find_domain(pos, stringa)
        print("--------- Domain:", domain, "------------")
        #ora punto all'apice del dominio
        pos += 3
        #punto all'inizio della parentesi quadra dove dovrebbe esserci lo shingle
        if stringa[pos] == "[" and stringa[pos+1] != "]":
            list_cluster = split_cluster(pos, stringa)
            return domain, list_cluster
        else:
            return domain, None #il dominio non possiende dei cluster

def buildDict(clusters):
    domain_cluster = {}
    for elem in clusters:
        #inizio al ricerca
        domain, list_cluster = init_find(elem)
        if list_cluster != None:
            domain_cluster[domain] = list_cluster
            print("\n")
        else:
            print("no cluster found \n")
    
    return domain_cluster
            
def Load_Data():
    clusters = []
    with open('dataset/FoxLink/part-00000', 'r') as file:
        for line in file:
            clusters.append(line.rstrip('\n'))
    with open('dataset/FoxLink/part-00002', 'r') as file:
        for line in file:
            clusters.append(line.rstrip('\n'))
    with open('dataset/FoxLink/part-00003', 'r') as file:
        for line in file:
            clusters.append(line.rstrip('\n'))
    return clusters

# Support method for create csv file

In [31]:
#creazione csv con i risultati delle feature
def built_CSV(domain_cluster):
    l = []
    for domain in domain_cluster.keys():
        clusters = domain_cluster[domain]
        for cluster in clusters:
            list_elem = [domain, cluster["shingle"], cluster["label"]]
            l.append(list_elem)
    
    df = pd.DataFrame(l, columns= ['domain', 'shingle', 'label'])
    df.to_csv('dataset/FoxLink/label_result.csv', index = False)

## Main

In [32]:
#Carica i dati
clusters = Load_Data()
#Creazione del dizionario con gli elementi
domain_cluster = buildDict(clusters)

--------- Domain: https://www.whatmobile.com.pk ------------
no cluster found 

--------- Domain: https://www.price.ru ------------
no cluster found 

--------- Domain: http://www.fountainpenscollectionforgiftsanyday1989.info ------------
- Find Shingle 1: [-1, 4, 8, 0, 7, 0, -1, 5]
- Find the url of cluster 1


--------- Domain: https://www.mymemory.co.uk ------------
- Find Shingle 1: [0, 3, 0, 1, 0, -1, 0, -1]
- Find the url of cluster 1
- Find Shingle 2: [0, 3, -1, 0, 0, 1, 0, -1]
- Find the url of cluster 2


--------- Domain: https://www.rcwilley.com ------------
- Find Shingle 1: [-1, 1, 2, 0, 0, 0, -1, 0]
- Find the url of cluster 1


--------- Domain: http://www.thesteilacoompub.com ------------
no cluster found 

--------- Domain: https://www.androidleo.com ------------
- Find Shingle 1: [0, 1, 3, -1, 1, 0, -1, 0, 3]
- Find the url of cluster 1
- Find Shingle 2: [0, -1, 2, -1, 7, 6, 2, 3]
- Find the url of cluster 2


--------- Domain: https://www.manualzz.com ------------
- 

In [33]:
#quando abbiamo due cluster calcolo dei pesi dei vari siti tramite la prima feature
print("=========== Inizio 1° Feature =========")
first_feature(domain_cluster)

=========== Inizio 1° Feature =========
---- Domain: https://www.mymemory.co.uk ----
- 1° Cluster
Time: 1.897  min
- 2° Cluster
Time: 2.273  min
---- Domain: https://www.androidleo.com ----
- 1° Cluster
Time: 0.613  min
- 2° Cluster
Time: 3.536  min
---- Domain: https://www.manualzz.com ----
- 1° Cluster
Time: 2.668  min
- 2° Cluster
Time: 0.217  min
---- Domain: https://www.addrom.com ----
- 1° Cluster
Time: 27.942  min
- 2° Cluster
Time: 7.315  min
---- Domain: https://www.etuo.de ----
- 1° Cluster
Time: 1.248  min
- 2° Cluster
Time: 2.693  min
---- Domain: https://www.bluechip.hu ----
- 1° Cluster
Time: 3.81  min
- 2° Cluster
Time: 2.515  min
---- Domain: https://www.youtab.pl ----
- 1° Cluster
Time: 1.847  min
- 2° Cluster
Time: 9.015  min
---- Domain: https://www.notebookcheck.org ----
- 1° Cluster
Time: 3.491  min
- 2° Cluster
Time: 0.989  min
---- Domain: https://www.laptopdirect.co.za ----
- 1° Cluster
Time: 0.431  min
- 2° Cluster
Time: 0.796  min
---- Domain: https://www.xia-

In [45]:
#con un cluster solo usiamo la seconda feature. Ritorna l'etichetta da assegnare al cluster
print("=========== Inizio 2° Feature =========")
second_feature(domain_cluster)

=========== Inizio 2° Feature =========
Domain: http://www.fountainpenscollectionforgiftsanyday1989.info
Tempo: 14.977  min
Domain: https://www.rcwilley.com
Tempo: 37.78  min
Domain: http://www.4k.com
Tempo: 2.09  min
Domain: https://www.samsungusbdrivers.net
Tempo: 0.68  min
Domain: https://www.productz.com
Tempo: 0.821  min
Domain: http://www.wholesalesalestore.com
Tempo: 6.439  min
Domain: https://www.youtube.com
Tempo: 0.805  min
Domain: http://www.icheckstock.com
Tempo: 0.879  min
Domain: https://www.kliklak.rs
Tempo: 34.114  min
Domain: https://www.alzashop.com
Tempo: 3.759  min
Domain: http://www.murahprice.org
Tempo: 0.454  min
Domain: https://www.redcorp.com
Tempo: 2.194  min
Domain: https://www.mediamart.vn
Tempo: 6.371  min
Domain: https://www.smithscity.co.nz
Tempo: 9.532  min


In [46]:
#creazione csv con i risultati delle feature
built_CSV(domain_cluster)

In [65]:
data = pd.read_csv("dataset/FoxLink/label.csv")
data2 = pd.read_csv("dataset/FoxLink/label_result.csv")
pd.concat([data, data2], axis=1, sort=False)

,domain,shingle,label,domain,shingle,label
0,http://www.fountainpenscollectionforgiftsanyda...,"[-1, 4, 8, 0, 7, 0, -1, 5]",Catalogo,http://www.fountainpenscollectionforgiftsanyda...,"[-1, 4, 8, 0, 7, 0, -1, 5]",Catalogo
1,https://www.mymemory.co.uk,"[0, 3, -1, 0, 0, 1, 0, -1]",Catalogo,https://www.mymemory.co.uk,"[0, 3, 0, 1, 0, -1, 0, -1]",Catalogo
2,https://www.rcwilley.com,"[-1, 1, 2, 0, 0, 0, -1, 0]",Prodotto,https://www.mymemory.co.uk,"[0, 3, -1, 0, 0, 1, 0, -1]",Prodotto
3,http://www.wholesalesalestore.com,"[2, 3, 0, 0, 3, -1, 7, -1]",Prodotto,https://www.rcwilley.com,"[-1, 1, 2, 0, 0, 0, -1, 0]",Prodotto
4,http://www.icheckstock.com,"[-1, 3, 7, 4, 4, 2, 6, 0]",Prodotto,https://www.androidleo.com,"[0, 1, 3, -1, 1, 0, -1, 0, 3]",Prodotto
5,https://www.etuo.de,"[-1, 2, 2, 1, 0, 1, -1, 4]",Catalogo,https://www.androidleo.com,"[0, -1, 2, -1, 7, 6, 2, 3]",Catalogo
6,https://www.etuo.de,"[0, -1, 0, -1, 0, 1, 5, 4]",Prodotto,https://www.manualzz.com,"[-1, 4, 6, 1, 3, 3, 1, 1, 5, 3]",Prodotto
7,https://www.kliklak.rs,"[-1, 16, 6, 4, 15, 3, 3, 13]",Catalogo,https://www.manualzz.com,"[-1, 0, 6, 0, 5, 6, 2, 3]",Catalogo
8,https://www.alzashop.com,"[0, 3, -1, -1, 0, 2, 3, 1]",Prodotto,https://www.addrom.com,"[-1, -1, 1, 3, 0, 6, 0, 5]",Catalogo
9,http://www.murahprice.org,"[-1, 8, 3, 3, 0, 4, 2, 3]",Prodotto,https://www.addrom.com,"[-1, 3, 1, 1, 0, 3, 0, -1]",Prodotto


# Metrica di accuracy applicata su foxlink

In [6]:
#ora avviamo il metodo che calcola l'accuratezza dell'algoritmo di cluster_labeling
metric('dataset/FoxLink/label.csv','dataset/FoxLink/label_foxlink.csv')

'Accuracy is: 45.0%'

# Accuracy applicata sul miglioramento dell'algoritmo

In [7]:
#ora avviamo il metodo che calcola l'accuratezza dell'algoritmo di cluster_labeling
metric('dataset/FoxLink/label.csv','dataset/FoxLink/label_result.csv')

'Accuracy is: 60.0%'

# Metrica applicata solo siti con due cluster

In [8]:
#ora avviamo il metodo che calcola l'accuratezza dell'algoritmo di cluster_labeling
metric('dataset/FoxLink/label2Cluster.csv','dataset/FoxLink/label_result.csv')

'Accuracy is: 90.9090909090909%'